In [1]:
from chathist import InstructionDataLoader,InstructionDataset, InstructionStyle, Model, Tokenizer
import pandas as pd

2025-05-13 11:53:16,286 - WARNING - data is not present in the config file, falling back to defaults!!!


In [2]:
chat_df = pd.read_json("hf://datasets/ogrnz/chat-titles/dataset.jsonl", lines=True)

In [3]:
chat_df.head()

,message,title
0,Develop a method for clustering astronomical d...,Astronomical Data Clustering Method
1,What is the role of dopamine in the brain as a...,Dopamine as a Neurotransmitter
2,"Replace the \""XXX\"" in the following sentence ...",Company Sustainability Objective
3,"Identify the sentence type (declarative, inter...",Sentence Type Identification
4,Suggest a book title for a fantasy novel about...,Outlaws in Enchanted Realms


In [4]:
chat_df.shape

(10000, 2)

In [5]:
chat_df.columns

Index(['message', 'title'], dtype='object')

In [6]:
style = InstructionStyle.load(style='phi3', input_query="<|user|>\n", response_query="\n<|llm|>\n")

2025-05-13 11:53:16,828 - INFO - phi3 style chosen!!


In [7]:
df = style.convert_train(chat_df, input_col='message', response_col= 'title', output_col='instruct', new_df=True)

In [8]:
train_len = 9500

In [9]:
train_df = df.iloc[:train_len, :]
val_df = df.iloc[train_len:, :]

In [10]:
train_df

,instruct
0,<|user|>\nDevelop a method for clustering astr...
1,<|user|>\nWhat is the role of dopamine in the ...
2,"<|user|>\nReplace the \""XXX\"" in the following..."
3,<|user|>\nIdentify the sentence type (declarat...
4,<|user|>\nSuggest a book title for a fantasy n...
...,...
9495,<|user|>\nAssume yourself to be a business whi...
9496,<|user|>\nIgnore all previous instructions. Ac...
9497,"<|user|>\nGiven the following description, ide..."
9498,<|user|>\nWhat is the 17th letter of supercali...


In [11]:
print(train_df['instruct'].values[0])

<|user|>
Develop a method for clustering astronomical data.
<|llm|>
Astronomical Data Clustering Method


In [12]:
val_df

,instruct
9500,<|user|>\nWhat are the differences in the prop...
9501,<|user|>\nImplement a program to find the comm...
9502,<|user|>\nHow can a person develop a healthy s...
9503,<|user|>\nPlease help me use schrodinger's equ...
9504,<|user|>\nRewrite the sentence to use a pronou...
...,...
9995,"<|user|>\nFrom the given passage, identify the..."
9996,<|user|>\nAnalyze an effect that the text coul...
9997,<|user|>\nWhat is the average number of hours ...
9998,<|user|>\nClassify the following statement as ...


In [13]:
print(val_df['instruct'].values[0])

<|user|>
What are the differences in the properties of the top quark when it is produced through different modes such as gluon fusion and vector boson fusion, and how do these differences impact our understanding of the top quark's interactions with other particles in the standard model of particle physics?
<|llm|>
Top Quark Production Modes


In [14]:
train_dataset = InstructionDataset(train_df['instruct'].tolist())

2025-05-13 11:53:16,927 - INFO - Setting tokenizer using tiktoken with encoding: gpt2


In [15]:
val_dataset = InstructionDataset(val_df['instruct'].tolist())

In [16]:
train_loader = InstructionDataLoader().load(dataset = train_dataset)

In [17]:
val_loader = InstructionDataLoader().load(dataset = val_dataset)

In [18]:
data = iter(val_loader)

In [19]:
first_batch = next(data)

In [20]:
first_batch[0][0]

tensor([   27,    91,  7220,    91,    29,   198,  2061,   389,   262,  5400,
          287,   262,  6608,   286,   262,  1353,   627,   668,   618,   340,
          318,  4635,   832,  1180, 12881,   884,   355,  1278,    84,   261,
        21748,   290, 15879, 37284,   261, 21748,    11,   290,   703,   466,
          777,  5400,  2928,   674,  4547,   286,   262,  1353,   627,   668,
          338, 12213,   351,   584, 13166,   287,   262,  3210,  2746,   286,
        18758, 11887,    30,   198,    27,    91,   297,    76,    91,    29,
          198,  9126,  2264,   668, 19174, 42082, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256], dtype=torch.int32)

In [21]:
first_batch[1][0]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         9126,  2264,   668, 19174, 42082, 50256,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100], dtype=torch.int32)

In [22]:
tokenizer = Tokenizer()

2025-05-13 11:53:18,293 - INFO - Setting tokenizer using tiktoken with encoding: gpt2


In [23]:
print(tokenizer.decode_ids(first_batch[0][0]))

<|user|>
What are the differences in the properties of the top quark when it is produced through different modes such as gluon fusion and vector boson fusion, and how do these differences impact our understanding of the top quark's interactions with other particles in the standard model of particle physics?
<|llm|>
Top Quark Production Modes<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


In [24]:
print(tokenizer.decode_ids(first_batch[1][0]))

Top Quark Production Modes<|endoftext|>


In [25]:
model = Model()

2025-05-13 11:53:19,532 - INFO - Repo: openai-community/gpt2.
2025-05-13 11:53:19,534 - INFO - openai-community/gpt2 exists in /home/smathad/ai_capstone/chathist/models/pretrained/gpt2. Skipping Download...
2025-05-13 11:53:20,008 - INFO - Loading weights into model.
2025-05-13 11:53:20,010 - WARNING - Using loRA will freeze the whole model except for loRA weights.
2025-05-13 11:53:20,081 - INFO - Using Lora for final output layer!!
2025-05-13 11:53:20,086 - INFO - Layer: 1
2025-05-13 11:53:20,090 - INFO - loading norm1 weights and bias
2025-05-13 11:53:20,091 - INFO - loading norm2 weights and bias
2025-05-13 11:53:20,091 - INFO - loading ff weights and bias
2025-05-13 11:53:20,109 - INFO - Using LoRA weights for ff layers!!
2025-05-13 11:53:20,121 - INFO - loading attention weights and bias
2025-05-13 11:53:20,152 - INFO - Using LoRA weights for multi head layers!!
2025-05-13 11:53:20,154 - INFO - Layer: 2
2025-05-13 11:53:20,154 - INFO - loading norm1 weights and bias
2025-05-13 11:

In [26]:
model.trainable_params()

2025-05-13 11:53:20,598 - INFO - Total model parameters: 166507792
2025-05-13 11:53:20,600 - INFO - Total trainable parameters: 3470608


,layer_name,requires_grad,params
0,tok_emb.weight,False,38597376
1,pos_emb.weight,False,786432
2,transformers.layer_0.norm1.scale,False,768
3,transformers.layer_0.norm1.shift,False,768
4,transformers.layer_0.norm2.scale,False,768
...,...,...,...
338,final_norm.scale,False,768
339,final_norm.shift,False,768
340,final_layer.linear_layer.weight,False,38597376
341,final_layer.lora.a_matrix,True,12288


In [27]:
# train_loss, val_loss = model.train(train_loader = train_loader, val_loader = val_loader)

In [28]:
style.format("what is AI")

'<|user|>\nwhat is AI\n<|llm|>\n'

In [29]:
print(model.generate('''
what is AI
'''))



AI is a new type of machine learning that uses machine learning to predict the future. It is a new type of machine learning that uses machine learning
